### Подготовим классификатор интентов

In [4]:
!pip install pymorphy2 stop_words annoy datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 8.8 MB/s eta 0:00:00


In [81]:
import catboost

import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from tqdm import tqdm_notebook

import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [5]:
from datasets import load_dataset

dataset = load_dataset("blinoff/medical_qa_ru_data", split='train[:1%]')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset medical_qa_ru_data downloaded and prepared to /root/.cache/huggingface/datasets/blinoff___medical_qa_ru_data/default/0.0.0/551476392fb5023cebc55b480ef3948c0ad2287c0ef0ad158a56fbb0bc37783f. Subsequent calls will reuse this data.


In [13]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [9]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [24]:
DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/GB_NLP/Chat_Bot/Data/'

In [31]:
assert True

# Preprocess for models fitting

sentences_answer = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0
max_rows = 1000

with open(DATA_PATH + "Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin, total=max_rows):
        spls = preprocess_txt(line)
        sentences_answer.append(spls)
        c += 1
        if c > max_rows:
            break

<ipython-input-31-e4c3dbf501bf>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin, total=max_rows):


  0%|          | 0/1000 [00:00<?, ?it/s]

In [32]:
assert True

# Preprocess for models fitting

sentences_med = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0
max_rows = 1000

for line in tqdm_notebook(dataset['desc'], total=max_rows):
      spls = preprocess_txt(line)
      sentences_med.append(spls)
      c += 1
      if c > max_rows:
          break

<ipython-input-32-aa2829444fb3>:13: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(dataset['desc'], total=max_rows):


  0%|          | 0/1000 [00:00<?, ?it/s]

In [73]:
df_med = pd.DataFrame(pd.Series([" ".join(_) for _ in sentences_med if len(_)>2]), columns=['text'])
df_med['class'] = 1
df_med.head(3)

,text,class
0,ларипронт 20 талетокчерез 23 часаочень понрави...,1
1,здравствуйте 78 неделя беременность 6 неделя а...,1
2,здравствуйте месячный должный прийти 23 март ф...,1


In [72]:
df_answ = pd.DataFrame(pd.Series([" ".join(_) for _ in sentences_answer if len(_)>2]), columns=['text'])
df_answ['class'] = 0
df_answ.head(3)

,text,class
0,вопрос тдв отдыхать лично советовать завести,0
1,мужик йопарить собачка 50 кошка,0
2,пол памытьbr таг тип каво,0


In [79]:
len(df_answ), len(df_med)

(656, 915)

In [74]:
df_all = pd.concat([df_answ, df_med])

In [94]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_all['text'], np.array(df_all['class']), test_size=0.2, stratify=df_all['class'],
                                                    random_state=42)

In [96]:
vectorizer = CountVectorizer(ngram_range=(1, 2))

In [100]:
from catboost import CatBoostClassifier
cat = CatBoostClassifier(verbose=False)
x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

cat_new = cat.fit(x_train_vec, y_train)

In [111]:
input_txt = preprocess_txt('Головокружение и боль')
vect = vectorizer.transform([" ".join(input_txt)])
prediction = cat_new.predict_proba(vect)
print(prediction)

[[0.4551052 0.5448948]]


In [102]:
from sklearn.metrics import accuracy_score
accuracy_score(y_true=y_test, y_pred=cat_new.predict(x_test_vec))

0.8666666666666667

Модель определяет медицинскую тематику с точность 86%

In [103]:
MODELS_PATH = '/content/drive/MyDrive/Colab Notebooks/GB_NLP/Chat_Bot/Models/'

In [104]:
import pickle
with open(MODELS_PATH+'cat_clsf.pkl', 'wb') as f:
   pickle.dump(cat_new, f)

In [106]:
with open(MODELS_PATH+'vectorizer.pk', 'wb') as fin:
  pickle.dump(vectorizer, fin)